In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error

from sklearn import neighbors
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
import json
with open("df_1hour_Feb.json", 'r', encoding='UTF-8') as f:
    F_data = json.load(f)

In [3]:
with open("df_1hour_Mar_05.json", 'r', encoding='UTF-8') as f:
    M_data = json.load(f)

In [4]:
COLUMNS = ['name', 'code', 'time', 'price', 'time_1', 'price_1', 'price_dif_1', 'sell_1', 'buy_1', 'volume_1', 'variation_1', 'post_num_1', 'unique_id_1', 'click_1', 'like_1', 'dislike_1', 'time_2', 'price_2', 'price_dif_2', 'sell_2', 'buy_2', 'volume_2', 'variation_2', 'post_num_2', 'unique_id_2', 'click_2', 'like_2', 'dislike_2', 'time_3', 'price_3', 'price_dif_3', 'sell_3', 'buy_3', 'volume_3', 'variation_3', 'post_num_3', 'unique_id_3', 'click_3', 'like_3', 'dislike_3', 'mkt_cap', 'kospi', 'kosdaq', 'trash', 'yesterday_closing_price', 'is_maximum', 'is_minimum', 'price_volatility', 'price_trend', 'average_price_volatility', 'sell_minus_buy_1', 'sell_minus_buy_2', 'sell_minus_buy_3', 'is_price_gap_stable', 'price_gap_volatility', 'is_like_higher', 'volume_trend', 'post_num_trend', 'unique_id_trend', 'click_trend', 'price_increase', 'did_price_increase', 'did_price_033', 'did_price_100', 'did_price_150', 'kospi_ind', 'kosdaq_ind', 'time_slot', 'ko_inter', 'early_mor', 'morning', 'lunch', 'afternoon', 'late', 'mkt_change', 'alpha', 'per_now', 'kospi_1', 'kospi_2', 'kospi_3', 'kospi_answer', 'kosdaq_1', 'kosdaq_2', 'kosdaq_3', 'kosdaq_answer', 'kospi_trend', 'kosdaq_trend', 'kospi_increase', 'kosdaq_increase', 'market_increase', 'did_opening_price_increase', 'price_1_sq', 'price_dif_1_sq', 'sell_1_sq', 'buy_1_sq', 'volume_1_sq', 'variation_1_sq', 'post_num_1_sq', 'unique_id_1_sq', 'click_1_sq', 'like_1_sq', 'dislike_1_sq', 'price_2_sq', 'price_dif_2_sq', 'sell_2_sq', 'buy_2_sq', 'volume_2_sq', 'variation_2_sq', 'post_num_2_sq', 'unique_id_2_sq', 'click_2_sq', 'like_2_sq', 'dislike_2_sq', 'price_3_sq', 'price_dif_3_sq', 'sell_3_sq', 'buy_3_sq', 'volume_3_sq', 'variation_3_sq', 'post_num_3_sq', 'unique_id_3_sq', 'click_3_sq', 'like_3_sq', 'dislike_3_sq', 'mkt_cap_sq', 'yesterday_closing_price_sq', 'price_volatility_sq', 'price_trend_sq', 'average_price_volatility_sq', 'sell_minus_buy_1_sq', 'sell_minus_buy_2_sq', 'sell_minus_buy_3_sq', 'price_gap_volatility_sq', 'volume_trend_sq', 'post_num_trend_sq', 'unique_id_trend_sq', 'click_trend_sq', 'kospi_ind_sq', 'kosdaq_ind_sq', 'time_slot_sq', 'ko_inter_sq', 'mkt_change_sq', 'alpha_sq', 'per_now_sq', 'kospi_1_sq', 'kospi_2_sq', 'kospi_3_sq', 'kosdaq_1_sq', 'kosdaq_2_sq', 'kosdaq_3_sq', 'kospi_trend_sq', 'kosdaq_trend_sq']

In [5]:
df_F = pd.DataFrame(F_data, columns = COLUMNS)
df_M = pd.DataFrame(M_data, columns = COLUMNS)
df = pd.concat([df_F, df_M])

In [6]:
df = df.dropna(axis=0, how='any')

In [26]:
time_filter_train = (df['time'].str.startswith("2018-02-21")) | \
                    (df['time'].str.startswith("2018-02-20")) | \
                    (df['time'].str.startswith("2018-02-14"))
        
time_filter_test = (df['time'].str.startswith("2018-02-22")) | \
                   (df['time'].str.startswith("2018-02-23")) | \
                   (df['time'].str.startswith("2018-02-26")) 

time_filter_pred = (df['time'].str.startswith("2018-02-28")) | \
                   (df['time'].str.startswith("2018-03-02")) | \
                   (df['time'].str.startswith("2018-03-05")) | \
                   (df['time'].str.startswith("2018-02-27"))
        
train_df = df[time_filter_train].reset_index(drop = True)
test_df = df[time_filter_test].reset_index(drop = True)
predict_df = df[time_filter_pred].reset_index(drop = True)

In [27]:
X_COL = ['price_1','price_dif_1','sell_1','buy_1','volume_1','variation_1','post_num_1','unique_id_1','click_1',
     'like_1','dislike_1','price_2','price_dif_2','sell_2','buy_2','volume_2','variation_2','post_num_2','unique_id_2',
     'click_2','like_2','dislike_2','price_3','price_dif_3','sell_3','buy_3','volume_3','variation_3','post_num_3',
     'unique_id_3','click_3','like_3','dislike_3','mkt_cap','kospi','kosdaq','trash','yesterday_closing_price',
     'is_maximum','is_minimum','price_volatility','price_trend','average_price_volatility','sell_minus_buy_1',
     'sell_minus_buy_2','sell_minus_buy_3','is_price_gap_stable','price_gap_volatility','is_like_higher',
     'volume_trend','post_num_trend','unique_id_trend','click_trend','kospi_ind','kosdaq_ind','time_slot',
     'ko_inter','early_mor','morning','lunch','afternoon','late','mkt_change','alpha','per_now','kospi_1','kospi_2',
     'kospi_3','kosdaq_1','kosdaq_2','kosdaq_3','kospi_trend','kosdaq_trend','did_opening_price_increase',
     'price_1_sq','price_dif_1_sq','sell_1_sq','buy_1_sq','volume_1_sq','variation_1_sq','post_num_1_sq',
     'unique_id_1_sq','click_1_sq','like_1_sq','dislike_1_sq','price_2_sq','price_dif_2_sq','sell_2_sq',
     'buy_2_sq','volume_2_sq','variation_2_sq','post_num_2_sq','unique_id_2_sq','click_2_sq','like_2_sq',
     'dislike_2_sq','price_3_sq','price_dif_3_sq','sell_3_sq','buy_3_sq','volume_3_sq','variation_3_sq',
     'post_num_3_sq','unique_id_3_sq','click_3_sq','like_3_sq','dislike_3_sq','mkt_cap_sq',
     'yesterday_closing_price_sq','price_volatility_sq','price_trend_sq','average_price_volatility_sq',
     'sell_minus_buy_1_sq','sell_minus_buy_2_sq','sell_minus_buy_3_sq','price_gap_volatility_sq',
     'volume_trend_sq','post_num_trend_sq','unique_id_trend_sq','click_trend_sq','kospi_ind_sq','kosdaq_ind_sq',
     'time_slot_sq','ko_inter_sq','mkt_change_sq','alpha_sq','per_now_sq','kospi_1_sq','kospi_2_sq','kospi_3_sq',
     'kosdaq_1_sq','kosdaq_2_sq','kosdaq_3_sq','kospi_trend_sq','kosdaq_trend_sq']

In [40]:
X = df[X_COL]
y = df['did_price_033']
X_train = train_df[X_COL]
X_test = test_df[X_COL]
y_train = train_df['did_price_033']
y_test = test_df['did_price_033']
X_perf = predict_df[X_COL]
y_perf = predict_df['did_price_033']
y_test_in = test_df['price_increase']

In [29]:
X_train.shape

(5425, 135)

In [30]:
X_pred.shape

(4036, 135)

In [31]:
X_test.shape

(6204, 135)

In [32]:
def get_neighbour_mean(num):
    
    knn = neighbors.KNeighborsClassifier(n_neighbors = num)
    y_pred = knn.fit(X_train, y_train).predict(X_test)
    error = (y_test != y_pred).astype(int)
    rv = error[y_pred == 1].mean()
    
    return rv    

In [33]:
def get_best_neighbour(num):
    rv_min = 1
    min_id = 1
    for i in range(1, num):
        value = get_neighbour_mean(i)
        if (value != None) and (value < rv_min):
            min_id = i
            rv_min = value
        print(i, value)
    return min_id, rv_min

In [34]:
best_n = get_best_neighbour(100)

1 0.7782805429864253
2 0.7863013698630137
3 0.7871621621621622
4 0.7838983050847458
5 0.7759056444818871
6 0.7772585669781932
7 0.774384236453202
8 0.7567114093959731
9 0.7612107623318386
10 0.7428023032629558
11 0.7645569620253164
12 0.7612244897959184
13 0.7653958944281525
14 0.7668997668997669
15 0.7773851590106007
16 0.7564102564102564
17 0.773542600896861
18 0.7491638795986622
19 0.7536585365853659
20 0.7362637362637363
21 0.7415730337078652
22 0.7375
23 0.7547770700636943
24 0.7428571428571429
25 0.743421052631579
26 0.7317073170731707
27 0.7446808510638298
28 0.7754010695187166
29 0.7692307692307693
30 0.7602739726027398
31 0.7650273224043715
32 0.7459016393442623
33 0.7650602409638554
34 0.7619047619047619
35 0.7575757575757576
36 0.7280701754385965
37 0.76
38 0.7522123893805309
39 0.726027397260274
40 0.7570093457943925
41 0.7708333333333334
42 0.7473684210526316
43 0.7435897435897436
44 0.7875
45 0.7699115044247787
46 0.782051282051282
47 0.776595744680851
48 0.81159420289855

In [35]:
best_n

(67, 0.5)

In [72]:
knn = neighbors.KNeighborsClassifier(n_neighbors = 51)
y_pred = knn.fit(X_train, y_train).predict(X_test)

In [73]:
result_matrix = confusion_matrix(y_test, y_pred)
result_matrix

array([[4852,   44],
       [1290,   18]], dtype=int64)

In [74]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.79      0.99      0.88      4896
          1       0.29      0.01      0.03      1308

avg / total       0.68      0.78      0.70      6204



In [75]:
y_port = y_test_in[y_pred == 1]
y_port.mean()

0.06794331302419349

In [76]:
y_test_in.mean()

-0.10475636824010301

In [77]:
y_port.shape

(62,)

In [94]:
y_perf_in = predict_df['price_increase']
knn = neighbors.KNeighborsClassifier(n_neighbors = 51)
y_perf_pred = knn.fit(X_train, y_train).predict(X_perf)
y_port = y_perf_in[y_perf_pred == 1]

In [102]:
y_perf_in.mean()

-0.14849392740473225

In [95]:
y_port.mean()

-0.0751963179236842

In [96]:
y_port.shape

(38,)

In [97]:
print(classification_report(y_perf, y_perf_pred))

             precision    recall  f1-score   support

          0       0.78      0.99      0.87      3148
          1       0.21      0.01      0.02       888

avg / total       0.65      0.77      0.68      4036



In [138]:
KNN_mod = pd.DataFrame(y_perf_pred, columns = ['KNN'])

In [139]:
KNN_mod['KNN'].value_counts()

0    3998
1      38
Name: KNN, dtype: int64

In [140]:
KNN_mod.to_json('KNN_mod.json', orient='values')

KNN & Logistic

In [115]:
var_filter = ['kospi_trend_sq',
 'price_1',
 'average_price_volatility',
 'price_dif_1',
 'price_dif_3_sq',
 'sell_1']

In [125]:
knn = neighbors.KNeighborsClassifier(n_neighbors = 2)
y_pred = knn.fit(X_train[var_filter], y_train).predict(X_test[var_filter])

In [126]:
result_matrix = confusion_matrix(y_test, y_pred)
result_matrix

array([[4421,  475],
       [1151,  157]], dtype=int64)

In [127]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.79      0.90      0.84      4896
          1       0.25      0.12      0.16      1308

avg / total       0.68      0.74      0.70      6204



In [116]:
def get_neighbour_mean(num):
    
    knn = neighbors.KNeighborsClassifier(n_neighbors = num)
    y_pred = knn.fit(X_train[var_filter], y_train).predict(X_test[var_filter])
    error = (y_test != y_pred).astype(int)
    rv = error[y_pred == 1].mean()
    
    return rv    

In [117]:
def get_best_neighbour(num):
    rv_min = 1
    min_id = 1
    for i in range(1, num):
        value = get_neighbour_mean(i)
        if (value != None) and (value < rv_min):
            min_id = i
            rv_min = value
        print(i, value)
    return min_id, rv_min

In [119]:
best_n

(40, 0.6666666666666666)

In [118]:
best_n = get_best_neighbour(100)

1 0.7694787748522299
2 0.7515822784810127
3 0.7723855092276145
4 0.765625
5 0.7783549783549784
6 0.7579143389199255
7 0.7812828601472135
8 0.7671840354767184
9 0.7840616966580977
10 0.7742782152230971
11 0.7994186046511628
12 0.7760416666666666
13 0.8033898305084746
14 0.8138686131386861
15 0.806941431670282
16 0.8087649402390438
17 0.803763440860215
18 0.8009478672985783
19 0.8070175438596491
20 0.8028169014084507
21 0.8071428571428572
22 0.8137254901960784
23 0.804
24 0.8187134502923976
25 0.8026905829596412
26 0.8
27 0.8052631578947368
28 0.7596153846153846
29 0.7702702702702703
30 0.7931034482758621
31 0.8091603053435115
32 0.8148148148148148
33 0.84
34 0.8333333333333334
35 0.8288288288288288
36 0.6744186046511628
37 0.7714285714285715
38 0.6764705882352942
39 0.7719298245614035
40 0.6666666666666666
41 0.7346938775510204
42 0.76
43 0.7692307692307693
44 0.84
45 0.8421052631578947
46 0.8421052631578947
47 0.76
48 0.7777777777777778
49 0.7727272727272727
50 0.8
51 0.761904761904761

In [128]:
y_port = y_test_in[y_pred == 1]
y_port.mean()

-0.10092731268053805

In [129]:
y_test_in.mean()

-0.10475636824010301

In [130]:
y_port.shape

(129,)

In [131]:
KNN_LOG_mod = pd.DataFrame(y_pred, columns = ['KNN & LOG'])
KNN_LOG_mod['KNN & LOG'].value_counts()

0    10111
1      129
Name: KNN & LOG, dtype: int64

In [132]:
KNN_LOG_mod.to_json('KNN_LOG_mod.json', orient='values')